### Revenue with a 7-day moving average

In [0]:
%sql
WITH daily AS (
  SELECT
    Transaction_Date AS transaction_date,
    SUM(Purchase_Amount) AS revenue
  FROM default.ecommerce_transactions
  GROUP BY Transaction_Date
)
SELECT 
  transaction_date,
  revenue,
  ROUND(
    AVG(revenue) OVER (
      ORDER BY transaction_date
      ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ), 2
  ) AS revenue_7day_moving_avg
FROM daily
ORDER BY transaction_date LIMIT 10;


transaction_date,revenue,revenue_7day_moving_avg
2023-03-09,34211.91999999999,34211.92
2023-03-10,35190.87000000001,34701.4
2023-03-11,33670.82000000001,34357.87
2023-03-12,40101.43999999999,35793.76
2023-03-13,32031.160000000014,35041.24
2023-03-14,40219.52,35904.29
2023-03-15,28690.880000000005,34873.8
2023-03-16,30139.58999999999,34292.04
2023-03-17,31950.729999999996,33829.16
2023-03-18,30783.57999999999,33416.7


### Top product categories by revenue

In [0]:
%sql
SELECT
  Product_Category,
  ROUND(SUM(Purchase_Amount),2) AS total_revenue
FROM default.ecommerce_transactions
GROUP BY Product_Category
ORDER BY total_revenue DESC;

Product_Category,total_revenue
Sports,3195335.9
Toys,3185652.36
Books,3181897.3
Clothing,3171225.96
Electronics,3133965.04
Grocery,3123579.52
Home & Kitchen,3108945.78
Beauty,3057387.79


### Category Performance Summary

In [0]:
%sql
SELECT
  Product_Category,
  COUNT(*) AS total_purchases,
  COUNT(DISTINCT User_Name) AS unique_customers,
  ROUND(SUM(Purchase_Amount),2) AS revenue
FROM default.ecommerce_transactions
GROUP BY Product_Category
ORDER BY revenue DESC;

Product_Category,total_purchases,unique_customers,revenue
Sports,6312,100,3195335.9
Toys,6392,100,3185652.36
Books,6253,100,3181897.3
Clothing,6224,100,3171225.96
Electronics,6320,100,3133965.04
Grocery,6215,100,3123579.52
Home & Kitchen,6209,100,3108945.78
Beauty,6075,100,3057387.79


### Customer Segmentation (VIP / Loyal / Regular)

In [0]:
%sql
WITH cust AS (
  SELECT
    User_Name,
    COUNT(*) AS txn_count,
    SUM(Purchase_Amount) AS total_spent
  FROM default.ecommerce_transactions
  GROUP BY User_Name
),
ranked AS (
  SELECT
    *,
    NTILE(3) OVER (ORDER BY total_spent DESC) AS spend_bucket
  FROM cust
)
SELECT
  CASE
    WHEN spend_bucket = 1 THEN 'VIP'
    WHEN spend_bucket = 2 THEN 'Loyal'
    ELSE 'Regular'
  END AS customer_tier,
  COUNT(*) AS customers
FROM ranked
GROUP BY customer_tier
ORDER BY customers DESC;


customer_tier,customers
VIP,34
Loyal,33
Regular,33
